In [2]:
:dep graph-ds = { path = ".", default-features = false }
:dep anyhow = "1.0"
:dep h3o = "0.3.0"
// :dep plotters = "0.3.4"

In [3]:
use graph_ds::{Graph, Node, Edge, AStarResult};
use graph_ds::hexagon_graph::{cell_graph_from_mpk, h3_network_from_osm, h3_network_from_gtfs, cell::Cell, h3cell::H3Cell, osm::OSMLayer, OSMOptions};

use std::time::Instant;
use std::sync::{Arc, RwLock, RwLockWriteGuard};
use std::collections::BTreeSet;
use std::collections::hash_map::DefaultHasher;
use std::hash::{Hash, Hasher};

use std::convert::TryInto;


## Cell Graph

In [8]:
let mut graph : Graph<Cell> = cell_graph_from_mpk("../..resources/de_mirage_hexagons.mpk.br")?;
println!("Graph has {} nodes", graph.nr_nodes());
graph.node_hash()

No such file or directory (os error 2)


In [ ]:
fn heuristic(start_cell: &Cell, end_cell: &Cell) -> f64 {
    let dx = (start_cell.a - end_cell.a).abs();
    let dy = (start_cell.b - end_cell.b).abs();
    let dz = (start_cell.a + start_cell.b - end_cell.a - end_cell.b).abs();
    let dlayer = (start_cell.layer - end_cell.layer).abs();
    ((dx + dy + dz) as f64 / 2.0 + dlayer as f64) * (start_cell.radius * 2) as f64
}

In [ ]:
let start = graph.get_random_node().unwrap();
let end = graph.get_random_node().unwrap();

println!("Start: {:?}", start);
println!("End: {:?}", end);

Start: Cell { a: 33, b: -27, radius: 24, layer: 0 }
End: Cell { a: 0, b: -53, radius: 24, layer: 1 }


In [ ]:
let start = Cell { a: -43, b: 41, radius: 24, layer: 1 };
let end = Cell { a: 38, b: -9, radius: 24, layer: 0 };

In [ ]:
println!("BFS start: {:?}, end: {:?}", start, end);
let now = Instant::now();
let (bfs_path, bfs_distance) = graph.bfs(start, Some(end), &None)?;
println!("time: {:?} µs", now.elapsed().as_micros());
println!("distance: {:?}", bfs_distance[0]);
println!("path length: {:?}", bfs_path);

println!("---");

println!("AStar start: {:?}, end: {:?}", start, end);
let now = Instant::now();
let astar_result = graph.astar(start, Some(end), &None, None, heuristic)?;
println!("time: {:?} µs", now.elapsed().as_micros());
println!("distance: {:?}", astar_result.distances.first().unwrap());
println!("path length: {:?}", astar_result.path.unwrap().len());

BFS start: Cell { a: -43, b: 41, radius: 24, layer: 1 }, end: Cell { a: 38, b: -9, radius: 24, layer: 0 }
time: 1088 µs
distance: None
path length: None
---
AStar start: Cell { a: -43, b: 41, radius: 24, layer: 1 }, end: Cell { a: 38, b: -9, radius: 24, layer: 0 }


no path found


## OSM + GTFS H3 Graph

In [4]:
let mut osm_g : Graph<H3Cell> = h3_network_from_osm("../resources/copenhagen-processed.osm.pbf", &OSMOptions::default())?;
println!("Graph has {} nodes, hash: {}", osm_g.nr_nodes(), osm_g.node_hash());

processing osm pbf file: ../resources/copenhagen-processed.osm.pbf
converted OSM file into 490098 edges
Graph has 418149 nodes, hash: 14980095458091112916


In [5]:
let mut gtfs_g : Graph<H3Cell> = h3_network_from_gtfs("../resources/rejseplanen.zip")?;
println!("Graph has {} nodes, hash: {}", gtfs_g.nr_nodes(), gtfs_g.node_hash());

getting GTFS feed from ../resources/rejseplanen.zip
routes: 1631
Graph has 103525 nodes, hash: 11658248065950047293


In [6]:
let edge_weights = gtfs_g.edges.as_ref().read().unwrap().iter().flat_map(|(id, set)| {
    set.iter().map(|e| {
        let weight = e.weight.unwrap();
        (weight * 1000.0) as i64
    }).collect::<Vec<i64>>()
}).collect::<Vec<i64>>();

In [7]:
let mut count = 0;
edge_weights.iter().for_each(|w| {
    if w == &0 {
        count +=1;
    }
});
println!("{} edges have weight 0", count);

0 edges have weight 0


In [ ]:
osm_g.merge(&mut gtfs_g)

Ok(())

In [ ]:
//631051737789436415, 631049684202690047, 631049684202697215

let start = H3Cell {
    cell: 0x8c1f0598d763dff.try_into().unwrap(),
    layer: -1,
};

let end = H3Cell {
    cell: 0x8c1f059890183ff.try_into().unwrap(),
    layer: -1,
};

fn h(start_cell: &H3Cell, end_cell: &H3Cell) -> f64 {
    start_cell
        .cell
        .grid_distance(end_cell.cell)
        .unwrap_or(i32::MAX) as f64
}

osm_g.astar(start, Some(end), &None, None, h)

[backtrace] found start node


Ok(AStarResult { path: Some([H3Cell { cell: 15-405461535436777 (8c1f0598d763dff), layer: -1 }, H3Cell { cell: 15-405461535434777 (8c1f0598d7639ff), layer: -1 }, H3Cell { cell: 15-405461535435777 (8c1f0598d763bff), layer: -1 }, H3Cell { cell: 15-405461535435777 (8c1f0598d763bff), layer: -2 }, H3Cell { cell: 15-405461535431777 (8c1f0598d7633ff), layer: -2 }, H3Cell { cell: 15-405461535054777 (8c1f0598d7459ff), layer: -2 }, H3Cell { cell: 15-405461535050777 (8c1f0598d7451ff), layer: -2 }, H3Cell { cell: 15-405461535053777 (8c1f0598d7457ff), layer: -2 }, H3Cell { cell: 15-405461535016777 (8c1f0598d741dff), layer: -2 }, H3Cell { cell: 15-405461535012777 (8c1f0598d7415ff), layer: -2 }, H3Cell { cell: 15-405461535035777 (8c1f0598d743bff), layer: -2 }, H3Cell { cell: 15-405461535031777 (8c1f0598d7433ff), layer: -2 }, H3Cell { cell: 15-405461535354777 (8c1f0598d75d9ff), layer: -2 }, H3Cell { cell: 15-405461535355777 (8c1f0598d75dbff), layer: -2 }, H3Cell { cell: 15-405461535351777 (8c1f0598d75d

In [ ]:
let a = H3Cell {
    cell: 0x8c1f0582686e5ff.try_into().unwrap(),
    layer: -1,
};

let b = H3Cell {
    cell: 0x8c1f0582686edff.try_into().unwrap(),
    layer: -1,
};

let c = H3Cell {
    cell: 0x8c1f05902b4d1ff.try_into().unwrap(),
    layer: -1,
};

In [ ]:
let origins = vec![start, end];
let destinations = vec![a, b, c, end];

let res = osm_g.matrix_astar_distance(origins, Some(destinations), true, None, h);

println!("{:#?}", res);

{
    H3Cell {
        cell: 15-405461100301777 (8c1f059890183ff),
        layer: -1,
    }: Ok(
        [
            62.60832291157254,
            0.0,
        ],
    ),
    H3Cell {
        cell: 15-405461535436777 (8c1f0598d763dff),
        layer: -1,
    }: Ok(
        [
            60.23964679723234,
            6.9120152613999775,
        ],
    ),
}


In [ ]:
osm_g.node_map.as_ref().read().unwrap().get_by_left(&start)

Some(459416)

In [ ]:
// osm_g.edges.as_ref().read().unwrap().get(&439124)